In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from skl2onnx.helpers.onnx_helper import load_onnx_model
from stitchnet.stitchonnx.utils import Net
from glob import glob
from pathlib import Path

netsFiles = sorted(glob('_models/fragments/net*'))
nets = []
for i,netsFile in enumerate(netsFiles):
    fragmentFiles = sorted(glob(str(Path(netsFile)/'fragment*.onnx')))
    onnxFragments = []
    for fragmentFile in fragmentFiles:
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
    net1 = Net(onnxFragments, i)
    nets.append(net1)

In [3]:
# import torch
# torch.ones(1,2).cuda()

In [4]:
from stitchnet.stitchonnx.utils import load_cats_and_dogs_dset,convert_imagenet_to_cat_dog_label
from stitchnet.stitchonnx.utils import accuracy_score_model,accuracy_score_net,load_dl
from stitchnet.stitchonnx.utils import generate_networks, ScoreMapper
from stitchnet.stitchonnx.report import Report
from stitchnet.stitchonnx.utils import evalulate_stitchnet

from tqdm import tqdm
import torch
import numpy as np
import os
from collections import defaultdict
import hashlib
import random
import time

random.seed(51)
np.random.seed(24)
torch.manual_seed(77)

K = 4
STITCH_BATCH_SIZE = 32 # todo study the effect
MAX_DEPTH = 16
THRESOULD = 0
TOTAL_THRESOULD = 0.5

RESULT_NAME = f"{int(time.time())}_result_beans_CKA_BS_{STITCH_BATCH_SIZE}_MD_{MAX_DEPTH}_T_{THRESOULD}_TT_{TOTAL_THRESOULD}_K_{K}"

EVAL_BATCH_SIZE = 16

from stitchnet.stitchonnx.utils import load_hf_train_val_dset_with_test_split

dataset_train, dataset_val = load_hf_train_val_dset_with_test_split('beans', 
                                                                    train='validation', 
                                                                    val='test', 
                                                                    label='labels',
                                                                    seed=47)

# dl_score = load_dl(dataset_train, batch_size=STITCH_BATCH_SIZE)
# data_score,t = next(iter(dl_score))
# data_score = data_score.numpy()

Found cached dataset beans (/home/jupyter-steerapi/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791)


  0%|          | 0/3 [00:00<?, ?it/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Loading cached shuffled indices for dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791/cache-cac060a5360da545.arrow
Loading cached shuffled indices for dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791/cache-fb581327019d6d2b.arrow
Loading cached processed dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791/cache-5f0b2533f6c38720.arrow
Loading cached processed dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791/cache-940888201d588dc5.arrow
Lo

In [5]:
# dataset_train = dataset_train.shuffle(seed=47)
dataset_train['labels'][0:120]

tensor([1, 0, 0, 1, 1, 2, 0, 2, 2, 0, 0, 2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 1, 1, 2,
        0, 1, 1, 0, 0, 0, 2, 2, 1, 2, 1, 1, 0, 1, 2, 0, 0, 0, 1, 1, 1, 2, 2, 0,
        1, 1, 1, 0, 2, 0, 1, 1, 0, 2, 0, 0, 2, 2, 0, 0, 2, 2, 1, 2, 0, 2, 2, 1,
        2, 0, 1, 2, 0, 2, 2, 2, 1, 1, 0, 1, 0, 0, 0, 2, 0, 1, 2, 1, 2, 2, 1, 1,
        2, 1, 2, 2, 2, 1, 2, 0, 1, 0, 0, 2, 2, 1, 0, 0, 0, 0, 2, 0, 2, 2, 0, 1])

In [6]:
# dataset_train.add_column(name='label', column=dataset_train['labels'])
# dataset_val.add_column(name='label', column=dataset_val['labels'])

In [7]:
# from transformers import AutoProcessor
from tqdm import tqdm
# p = AutoProcessor.from_pretrained('microsoft/resnet-50')
inps = []
# tgts = []
# for x in tqdm(dataset_train.take(STITCH_BATCH_SIZE)):
for x in tqdm(dataset_train.select(range(STITCH_BATCH_SIZE))):
    try:
        inps += x['pixel_values']
        # inps += p(x['image'])['pixel_values']
        # tgts += [x['label']]
    except:
        pass
data_score = np.stack(inps)
data_score.shape

100%|██████████| 32/32 [00:01<00:00, 17.58it/s]


(32, 3, 224, 224)

In [8]:
# range(1)
k = 0
if os.path.exists(f'./_results/{RESULT_NAME}.txt'):
    with open(f'./_results/{RESULT_NAME}.txt', 'r') as f:
        k = len(f.read().split('\n'))

In [9]:
from tqdm import tqdm
from stitchnet.stitchonnx.report import Report, ReportPlants, ReportKNNHFDataset
import traceback
scoreMapper = ScoreMapper(nets, data_score, scoring_method='CKA')
with ReportKNNHFDataset(EVAL_BATCH_SIZE, f'./_results/{RESULT_NAME}.txt', 'a', label='labels') as report:
    # for _ in tqdm(range(50)):
    generator = generate_networks(nets, scoreMapper, data_score, 
                          threshold=THRESOULD, totalThreshold=TOTAL_THRESOULD, 
                          maxDepth=MAX_DEPTH, sample=False, K=K)
    for i,(s,net) in enumerate(generator):
        try:
            netname = f"_results/{RESULT_NAME}/net{k:03}"
            report.evaluate(nets, net, netname, s, dataset_val, dataset_train)
            net.save(netname)
            k += 1
        except Exception as e:
            print('ERROR', e)
            traceback.print_exc()
            pass

(32, 3, 224, 224)
(32, 64, 27, 27)
(32, 192, 13, 13)
(32, 384, 13, 13)
(32, 256, 13, 13)
(32, 9216)
(32, 4096)
(32, 4096)
current depth: 1
(32, 3, 224, 224)
(32, 3, 224, 224)
(32, 3, 224, 224)
(32, 3, 224, 224)
(32, 64, 56, 56)
(32, 64, 224, 224)
(32, 256, 56, 56)(32, 16, 56, 56)

(32, 256, 56, 56)
(32, 16, 56, 56)
(32, 64, 112, 112)
(32, 72, 28, 28)
(32, 512, 28, 28)
(32, 128, 112, 112)(32, 512, 28, 28)

(32, 24, 28, 28)(32, 128, 56, 56)

(32, 1024, 14, 14)
(32, 256, 56, 56)
(32, 96, 14, 14)
(32, 1024, 14, 14)
(32, 2048)(32, 256, 56, 56)
(32, 1024)(32, 40, 14, 14)


(32, 120, 14, 14)
(32, 256, 28, 28)
(32, 48, 14, 14)
(32, 512, 28, 28)(32, 288, 7, 7)

(32, 512, 28, 28)
(32, 96, 7, 7)
(32, 576)
(32, 1024)
(32, 512, 14, 14)
(32, 512, 14, 14)
(32, 512, 14, 14)
(32, 25088)
(32, 4096)
(32, 4096)
potential next fragments: 4
potential next fragments before thresholding of 0: 4 ['1.0', '0.67', '0.6', '0.56']
potential next fragments after filter duplicated fragments: 4 ['1.0', '0.67', '0.6', 

100%|██████████| 5/5 [00:00<00:00, 62.37it/s]


(133, 1000) (133,)


 25%|██▌       | 1/4 [00:01<00:03,  1.22s/it]

(32, 3, 224, 224)


...

Traceback (most recent call last):
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 1121, in recursive_stitching
    newcurr_fragment = stitch_fragments(curr, nextf, data)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 740, in stitch_fragments
    nw = adjust_w(tX, tY, w)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 641, in adjust_w
    return adjust_w_conv(tX, tY, w)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 617, in adjust_w_conv
    print('diff sampled', (acts1sampled - acts2sampled).pow(2).sum())
RuntimeError: The size of tensor a (256) must match the size of tensor b (64) at non-singleton dimension 1


ERROR The size of tensor a (64) must match the size of tensor b (256) at non-singleton dimension 1
totalscore 0.8316447138786316


Traceback (most recent call last):
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 1121, in recursive_stitching
    newcurr_fragment = stitch_fragments(curr, nextf, data)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 740, in stitch_fragments
    nw = adjust_w(tX, tY, w)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 641, in adjust_w
    return adjust_w_conv(tX, tY, w)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 617, in adjust_w_conv
    print('diff sampled', (acts1sampled - acts2sampled).pow(2).sum())
RuntimeError: The size of tensor a (64) must match the size of tensor b (256) at non-singleton dimension 1


diff sampled tensor(1.0251e+08)
epoch 0 loss 46.96131874590504 torch.Size([401408, 64]) torch.Size([401408, 64])
(32, 3, 224, 224)
current depth: 2
potential next fragments: 4
potential next fragments before thresholding of 0: 4 ['0.7', '0.56', '0.55', '0.5']
potential next fragments after filter duplicated fragments: 3 ['0.7', '0.56', '0.5']
potential next fragments after thresholding of 0: 3 ['0.7', '0.56', '0.5']
totalscore 0.581516745667976
diff sampled tensor(68836120.)
epoch 0 loss 132.4034359601079 torch.Size([401408, 128]) torch.Size([401408, 128])
(32, 3, 224, 224)
current depth: 3
potential next fragments: 4
potential next fragments before thresholding of 0: 4 ['0.71', '0.71', '0.53', '0.5']
potential next fragments after filter duplicated fragments: 2 ['0.71', '0.53']
potential next fragments after thresholding of 0: 2 ['0.71', '0.53']
totalscore 0.6421898007392883
diff sampled tensor(3019062.7500)
epoch 0 loss 1.8139722517558508 torch.Size([100352, 64]) torch.Size([100352, 

Traceback (most recent call last):
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 1121, in recursive_stitching
    newcurr_fragment = stitch_fragments(curr, nextf, data)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 740, in stitch_fragments
    nw = adjust_w(tX, tY, w)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 641, in adjust_w
    return adjust_w_conv(tX, tY, w)
  File "/home/jupyter-steerapi/stitchnet/stitchnet/stitchonnx/utils.py", line 617, in adjust_w_conv
    print('diff sampled', (acts1sampled - acts2sampled).pow(2).sum())
RuntimeError: The size of tensor a (64) must match the size of tensor b (256) at non-singleton dimension 1


In [10]:
# from stitchnet.stitchonnx.utils import accuracy_score_net_plants

In [11]:
# for x,t in dataset_val:
#     print(t)

In [12]:
# dataset_val[0]

In [13]:
# accuracy_score_net_plants(net, dataset_val, dataset_train)

In [14]:
# from sklearn.neighbors import KNeighborsClassifier
# knn = KNeighborsClassifier(n_neighbors=1)
# knn.fit(np.ones([100,10]), np.ones([100,1]))


In [15]:
# y = knn.predict(np.ones([1,10]))
# y

In [16]:
# dataset_val

In [17]:
print('DONE')

DONE
